In [1]:
from bs4 import BeautifulSoup
import urllib.request as urllib2
import requests
import pandas as pd
from pandas import Series, DataFrame
from ipywidgets import FloatProgress
from time import sleep
from IPython.display import display
import re
import pickle
import csv
import time
import os
import json
import numpy
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import math

## Preparing data

In [1]:
df = pd.read_csv(r"Path to csv")
options = webdriver.ChromeOptions()
options.headless = True

NameError: name 'pd' is not defined

In [3]:
df["GoogleUrl"] = "https://www.google.com/search?q="
for i in range(len(df.winery)):
    try:
        if math.isnan(df.region[i]):
            df.region[i] = " "
        df.GoogleUrl[i] = "https://www.google.com/search?q=" + (df.winery[i] + " winery in " + df.region[i].split(',')[0] + " "+ df.region[i].split(',')[df.region[i].count(',')] + " " + df.region[i].split(',')[df.region[i].count(',')-1]).replace('&','%26').replace(' ','%20')
    except:
        df.GoogleUrl[i] = "https://www.google.com/search?q=" + (df.winery[i] + " winery in " + df.region[i].split(',')[0] + " "+ df.region[i].split(',')[df.region[i].count(',')] + " " + df.region[i].split(',')[df.region[i].count(',')-1]).replace('&','%26').replace(' ','%20')

In [4]:
def create_url(string):
    return string.replace(" ","+")

# The Scrapper

In [5]:
%%time
list_of_google_wineries = []
list_of_non_google_wineries = []
index = []

start = input("Wineries from: ")
finish = input("To: ")

vector = []
header = ["winery","region","google title","adress","website","phone","rating","reviews","schedule","socialmedia","status","latitude","longitude","GoogleUrl","GoogleMapsUrl"]
header2 = ["winery","region","link"]
count = 0
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
for i in range(int(start),int(finish)):
    driver.get(df.GoogleUrl[i])
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR ,'#W0wltc > div'))).click()
    except:
        print("Asta nu cere")
    time.sleep(10)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source,"lxml")
   
    
    #=========================CHECK LINK===============================
    
    print(str(i)+". " +df.GoogleUrl[i])
    section = soup.find("div",{"class":"liYKde g VjDLd"})

    if section is not None:
        count= count+1
    else:
        row = [df.winery[i],df.region[i],df.GoogleUrl[i]]
        list_of_non_google_wineries.append(row)
        vector.append(i)
        continue
    
    #=========================TITLE===============================
    
    try:
        title = soup.find("h2",{"data-attrid":"title"}).get_text()
    except:
        title = "NULL"
    print(str(i)+". "+title)
    
    #=========================ADRESS===============================
    
    try:    
        adress = soup.find("span",{"class":"LrzXr"}).get_text()
    except:
        adress = "NULL"
    print("adress: "+adress)
    
    #=========================WEBSITE===============================
    
    try:
        website = soup.find("a",{"class":"ab_button"})["href"]
        if website == "#":
            website = "NULL"
    except:
        website="NULL"
    print("website: "+ website)
    
    #=========================PHONE===============================
    
    try:
        phone = soup.find("span",{"class":"LrzXr zdqRlf kno-fv"}).find("a").find("span").find("span").get_text()
    except:
        phone = "NULL"
    print("phone: "+phone)
    
    #=========================RATING===============================
    
    try:
        rate = soup.find("div",{"class":"Ob2kfd"}).find("span").get_text()
    except:
        rate = "NULL"
    print("rating: "+rate)
    
    #=========================REVIEWS===============================
    
    try:
        reviews = soup.find("span",{"class":"hqzQac"}).find("span",{"jscontroller":"qjk5yc"}).find("a").find("span").get_text()
    except:
        reviews="NULL"
    print("reviews: "+reviews)
    
    #=========================SCHEDULE===============================
    
    try:
        table = soup.find('div',{"class":"a-h"}).find("div",{"class":"vk_bk"}).find("table")
        days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
        schedule = ""
        for child in table.children:
            for td in child:
                for day in days:
                    if td.text.find(day)!= -1:
                        schedule = schedule + td.text[0:len(day)] + " "+ td.text[len(day):]+"\n"
    except:
        schedule = "NULL"
    print("schedule: "+schedule)
    
    #=========================SOCIALMEDIA===============================
    
    try:
        socialmedia = ""
        SocialMediaSec = soup.find_all("div",{"class":"PZPZlf dRrfkf kno-vrt-t"})
        for child in SocialMediaSec:
            socialmedia = socialmedia + child.find("div").find("g-link").find("a")["href"]+"\n"
        if socialmedia == "":
            socialmedia = "NULL"
    except:
        socialmedia = "NULL"
    print("socialmedia: "+socialmedia)
    
    #=========================STATUS===============================
    
    status=""
    try:
        status = soup.find("div",{"class":"fYOrjf kp-hc"}).find("div",{"class":"VTNskf"}).find("div",{"class":"b4cFMb"}).find("span",{"class":"hBA2d Shyhc"}).find("span").get_text()
    except:
        status = "OPEN"
    print("status: " + status)
    
    #=========================lONG&LAT===============================
    divs = ""
    x = ""
    
    #-------------------------SEARCHING FOR MAPS LINK--------------------------------
    
    try:
        print(str(i)+".")
        divs = soup.find("div",{"class":"Ftghae iirjIb DaSCDf"}).find("div",{"class":"SPZz6b"}).find("div",{"class":"IzNS7c duf-h"}).find("div",{"class":"QqG1Sd"})
        for j in range(3):
            try: 
                if divs.find("a")["data-url"].startswith("/maps/dir"):
                    x = divs.find("a")["data-url"]
                    break
            except:
                divs = divs.nextSibling
    except:
        try:
            x= soup.find("div",{"class":"d7sCQ kp-header"}).find("div",{"class":"wDYxhc NFQFxe oHglmf xzPb7d"}).find("div",{"class":"Ggdpnf kno-fb-ctx"}).find("div",{"class":"SwlyWb rhsmap4col"}).find("a")["data-url"]
            x= x.replace("place","dir/")
            x= x[:x.find("@")-1]
            x = x + "+"+ create_url(adress)
        except:
            row = [df.winery[i],df.region[i],df.GoogleUrl[i]]
            print("LA LINKU DE MAPS")
            list_of_non_google_wineries.append(row)
            continue
    mapsLink=""
    lat = ""
    long = ""
    
    #-------------------------GETTING THE LAT AND LONG--------------------------------
    MAP = ""
    try:
        mapsLink = "https://www.google.com" + x
        driver.get(mapsLink)
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR ,'#yDmH0d > c-wiz > div > div > div > div.NIoIEf > div.G4njw > div.AIC7ge > div.CxJub > div.VtwTSb > form:nth-child(2) > div > div > button > div.VfPpkd-RLmnJb'))).click()
        except:
            pass
        time.sleep(5)
       #for i in range(9):
            #print(i)
        #    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR ,'#widget-zoom-in > div'))).click()
        #time.sleep(2)
        #first = driver.current_url
        #second = first[first.find("@"):]
        #third = second[second.find(","):]
        #lat = second[1:second.find(",")]
        #long = third[1:third[1:].find(",")+1]
        MAP = driver.current_url
        long = driver.current_url[driver.current_url.find("!1d")+3:driver.current_url.find("!2d")]
        lat = driver.current_url[driver.current_url.find("!2d")+3:]
        if lat.startswith("tps://")!=0:
            index.append(i)
        print("Lat: " + lat)
        print("Long: " + long)
    except:
        print("LA MAPS")
        row = [df.winery[i],df.region[i],df.GoogleUrl[i]]
        list_of_non_google_wineries.append(row)
        continue
        
    #=========================ROW INSERT===============================
    
    #try:
    #header = ["winery","region","google title","adress","website","phone","rating","reviews","schedule","socialmedia","status","latitude","longtitude","GoogleUrl","GoogleMapsUrl"]
    if title != "NULL":
        row = [df.winery[i], df.region[i], title, adress, website, phone, rate, reviews, schedule, socialmedia,status,lat,long,df.GoogleUrl[i],MAP]
        list_of_google_wineries.append(row)
    else:
        row = [df.winery[i],df.region[i],df.GoogleUrl[i]]
        print("LA SFARSIT")
        list_of_non_google_wineries.append(row)
        

Wineries from: 10000
To: 12000
10000. https://www.google.com/search?q=Domaine%20de%20Coste%20Chaude%20winery%20in%20Côtes%20du%20Rhône%20Villages%20Visan%20%20France%20%20Rhône%20Valley
10000. Domaine de Coste Chaude
adress: 3100 chemin de la Carne, 84820 Visan, France
website: http://www.domaine-coste-chaude.com/
phone: +33 6 80 59 93 01
rating: 5.0
reviews: 12 Google reviews
schedule: Saturday Closed
Sunday Closed
Monday (Easter Monday)8 am–4 pmHours might differ
Tuesday 8 am–4 pm
Wednesday 8 am–4 pm
Thursday 8 am–4 pm
Friday 8 am–4 pm

socialmedia: https://fr.linkedin.com/in/vincent-tramier-8309371a8

status: OPEN
10000.
Lat: 44.3230965
Long: 4.979222
Asta nu cere
10001. https://www.google.com/search?q=Domaine%20de%20Courcel%20winery%20in%20Pommard%20%20France%20%20Burgundy
10001. Domaine De Courcel
adress: 29 Pl. de l'Église, 21630 Pommard, France
website: http://www.domainedecourcel-pommard.fr/
phone: +33 3 80 22 10 64
rating: 4.0
reviews: 4 Google reviews
schedule: NULL
socialmed

10017. https://www.google.com/search?q=Domaine%20de%20Fauterie%20winery%20in%20Saint-Joseph%20%20France%20%20Rhône%20Valley
10017. Domaine de Fauterie
adress: 10 Av. du Général Charles de Gaulle, 07130 Saint-Péray, France
website: NULL
phone: +33 4 75 80 04 25
rating: 4.8
reviews: 6 Google reviews
schedule: NULL
socialmedia: NULL
status: Permanently closed
10017.
Lat: 44.9497168
Long: 4.84406
Asta nu cere
10018. https://www.google.com/search?q=Domaine%20de%20Fenouillet%20winery%20in%20Beaumes-de-Venise%20%20France%20%20Rhône%20Valley
10018. FIELD FENOUILLET
adress: 123 All. St Roch, 84190 Beaumes-de-Venise, France
website: https://www.domaine-fenouillet.fr/
phone: +33 4 90 62 95 61
rating: 4.8
reviews: 31 Google reviews
schedule: Saturday 9 am–12:15 pm, 2:30–6:45 pm
Sunday Closed
Monday (Easter Monday)ClosedHoliday opening hours
Tuesday 9 am–12:15 pm, 2:30–6:45 pm
Wednesday 9 am–12:15 pm, 2:30–6:45 pm
Thursday 9 am–12:15 pm, 2:30–6:45 pm
Friday 9 am–12:15 pm, 2:30–6:45 pm

socialmedia:

Lat: 41.6970034
Long: 9.3808334
Asta nu cere
10035. https://www.google.com/search?q=Domaine%20de%20Grand%20Garant%20winery%20in%20Fleurie%20%20France%20%20Beaujolais
10035. Domaine Clos des Garands
adress: Les Garants, 69820 Fleurie, France
website: http://www.closdesgarands.fr/fr/accueil.html
phone: +33 4 74 69 80 01
rating: 4.5
reviews: 6 Google reviews
schedule: NULL
socialmedia: NULL
status: OPEN
10035.
Lat: 46.1944581
Long: 4.7103839
Asta nu cere
10036. https://www.google.com/search?q=Domaine%20de%20Grandmaison%20winery%20in%20Pessac-Léognan%20%20France%20%20Bordeaux
10036. Domaine de Grandmaison
adress: 182 Av. de la Duragne, 33850 Léognan, France
website: https://www.domaine-de-grandmaison.fr/
phone: +33 5 56 64 75 37
rating: 4.7
reviews: 21 Google reviews
schedule: Saturday 9 am–12 pm, 2:30–6 pm
Sunday Closed
Monday (Easter Monday)9 am–12 pm, 2–6:15 pmHours might differ
Tuesday 9 am–12 pm, 2–6:15 pm
Wednesday 9 am–12 pm, 2–6:15 pm
Thursday 9 am–12 pm, 2–6:15 pm
Friday 9 am–12 p

Lat: 44.8160912
Long: 0.0672787
Asta nu cere
10051. https://www.google.com/search?q=Domaine%20de%20Laballe%20winery%20in%20Côtes%20de%20Gascogne%20%20France%20%20Southwest%20France
10051. Domaine de Laballe
adress: Château de Laballe, 40310 Parleboscq, France
website: https://boutique.laballe.fr/fr/
phone: +33 5 58 73 81 57
rating: 4.5
reviews: 8 Google reviews
schedule: Saturday 10 am–1 pm
Sunday Closed
Monday (Easter Monday)Closed
Tuesday Closed
Wednesday 10 am–1 pm, 2–6 pm
Thursday Closed
Friday 10 am–1 pm, 2–6 pm

socialmedia: https://www.instagram.com/laballe_chateau/
https://fr.linkedin.com/company/famillelaudet

status: OPEN
10051.
Lat: 43.920251
Long: 0.015976
Asta nu cere
10052. https://www.google.com/search?q=Domaine%20de%20Lafayette%20winery%20in%20Brouilly%20%20France%20%20Beaujolais
Asta nu cere
10053. https://www.google.com/search?q=Domaine%20de%20Lalande%20winery%20in%20Mâcon%20Chaintré%20%20France%20%20Burgundy
10053. Castle Lavernette
adress: La Vernette, 71570 Leynes,

Lat: 43.631938
Long: 3.294884
Asta nu cere
10068. https://www.google.com/search?q=Domaine%20de%20Marcoux%20winery%20in%20Châteauneuf-du-Pape%20%20France%20%20Rhône%20Valley
10068. Domaine de Marcoux
adress: chemin de la 84100, 198 Chem. Gironde, 84100 Orange, France
website: http://www.domainedemarcoux.fr/
phone: +33 4 90 34 67 43
rating: 4.8
reviews: 9 Google reviews
schedule: NULL
socialmedia: NULL
status: OPEN
10068.
Lat: 44.1063528
Long: 4.8230388
Asta nu cere
10069. https://www.google.com/search?q=Domaine%20de%20Maubet%20winery%20in%20Côtes%20de%20Gascogne%20%20France%20%20Southwest%20France
Asta nu cere
10070. https://www.google.com/search?q=Domaine%20de%20Millet%20winery%20in%20Coteaux%20Bourguignons%20%20France%20%20Burgundy
Asta nu cere
10071. https://www.google.com/search?q=Domaine%20de%20Mirail%20winery%20in%20Côtes%20de%20Gascogne%20%20France%20%20Southwest%20France
10071. Domaine de Mirail
adress: Domaine de Mirail, 32700 Lectoure, France
website: https://www.domainedemira

...

Lat: 38.7113931
Long: -122.9782966
Asta nu cere
11903. https://www.google.com/search?q=Ferraton%20Père%20et%20Fils%20winery%20in%20Saint-Joseph%20%20France%20%20Rhône%20Valley
11903. Ferraton Père & Fils Caveau
adress: 7 Quai Arthur Rostaing, 26600 Tain-l'Hermitage, France
website: http://www.caveau-ferraton.fr/
phone: +33 4 75 08 51 93
rating: 4.6
reviews: 68 Google reviews
schedule: Sunday Closed
Monday (Easter Monday)Closed
Tuesday 10 am–12:30 pm, 2:30–7 pm
Wednesday 10 am–12:30 pm, 2:30–7 pm
Thursday 10 am–12:30 pm, 2:30–7 pm
Friday 10 am–12:30 pm, 2:30–7 pm
Saturday 10 am–12:30 pm, 2:30–7 pm

socialmedia: https://www.instagram.com/ferraton_rhone_wines/
https://www.facebook.com/ferratonpereetfils/

status: OPEN
11903.
Lat: 45.0684394
Long: 4.8401349
Asta nu cere
11904. https://www.google.com/search?q=Ferratus%20winery%20in%20Ribera%20del%20Duero%20%20Spain%20%20Northern%20Spain
Asta nu cere
11905. https://www.google.com/search?q=Ferraud%20et%20Fils%20winery%20in%20Brouilly%20%20Fra

Lat: 34.6659474
Long: -120.1151818
Asta nu cere
11920. https://www.google.com/search?q=Fest%20winery%20in%20Central%20Valley%20%20Chile%20Central%20Valley
11920. Maipo Valley Wine Tours
adress: Londres 70, Santiago, Región Metropolitana, Chile
website: http://www.maipo.travel/
phone: +56 9 9593 9103
rating: 4.1
reviews: 21 Google reviews
schedule: Saturday (Holy Saturday)10 am–6 pmHours might differ
Sunday 10 am–6 pm
Monday 10 am–6 pm
Tuesday 10 am–6 pm
Wednesday 10 am–6 pm
Thursday 10 am–6 pm
Friday 10 am–6 pm

socialmedia: NULL
status: OPEN
11920.
Lat: -33.4449337
Long: -70.648302
Asta nu cere
11921. https://www.google.com/search?q=Festival%20and%20Feast%20winery%20in%20Sonoma%20County%20%20US%20%20California
Asta nu cere
11922. https://www.google.com/search?q=Fetish%20winery%20in%20Barossa%20%20Australia%20%20South%20Australia
Asta nu cere
11923. https://www.google.com/search?q=Fetzer%20winery%20in%20California%20%20US%20%20California
11923. Fetzer Winery
adress: 13100 Old River Rd,

Lat: 44.724637
Long: 8.0886696
Asta nu cere
11942. https://www.google.com/search?q=Fichtenbauer-Mold%20winery%20in%20Kamptal%20%20Austria%20Kamptal
11942. Weingut Fichtenbauer-Mold
adress: Rosenhügelweg 12, 3550 Langenlois, Austria
website: http://fichtenbauer-mold.at/
phone: +43 2734 3714
rating: 5.0
reviews: 5 Google reviews
schedule: NULL
socialmedia: NULL
status: OPEN
11942.
Lat: 48.468868
Long: 15.6746236
Asta nu cere
11943. https://www.google.com/search?q=Fiddlebender%20winery%20in%20Arizona%20%20US%20%20Arizona
Asta nu cere
11944. https://www.google.com/search?q=Fiddlehead%20winery%20in%20Willamette%20Valley%20%20US%20%20Oregon
11944. Holloran Vineyard Wines
adress: 8795 NE Worden Hill Rd, Dundee, OR 97115, United States
website: http://www.holloranvineyardwines.com/
phone: +1 971-720-0632
rating: 5.0
reviews: 29 Google reviews
schedule: Saturday 11 am–5 pm
Sunday (Easter)11 am–5 pmHours might differ
Monday 11 am–5 pm
Tuesday 11 am–5 pm
Wednesday 11 am–5 pm
Thursday 11 am–5 pm
F

Lat: tps://www.google.com/maps/dir//Figge+Family+Vineyards++1+310-409-8077/@54.5512799,-4.4737716,5z
Long: tps://www.google.com/maps/dir//Figge+Family+Vineyards++1+310-409-8077/@54.5512799,-4.4737716,5
Asta nu cere
11969. https://www.google.com/search?q=Figgins%20winery%20in%20Walla%20Walla%20Valley%20(WA)%20%20US%20%20Washington
11969. FIGGINS Winery
adress: 2900 Melrose St, Walla Walla, WA 99362, United States
website: http://www.figginswine.com/
phone: +1 509-522-7808
rating: 4.7
reviews: 22 Google reviews
schedule: Saturday 11 am–4 pm
Sunday (Easter)11 am–4 pmHours might differ
Monday Closed
Tuesday Closed
Wednesday 11 am–4 pm
Thursday 11 am–4 pm
Friday 11 am–4 pm

socialmedia: https://www.instagram.com/figginswine/

status: OPEN
11969.
Lat: 46.0797784
Long: -118.2903507
Asta nu cere
11970. https://www.google.com/search?q=Fighter%20Pilot%20Red%20winery%20in%20Adelaida%20District%20%20US%20%20California
11970. Adelaida Vineyards
adress: 5805 Adelaida Rd, Paso Robles, CA 93446, Unite

Lat: -33.103585
Long: -68.671267
Asta nu cere
11993. https://www.google.com/search?q=Finca%20Albret%20winery%20in%20Navarra%20%20Spain%20%20Northern%20Spain
11993. Bodegas Finca Albret
adress: 31515 Cadreita, Navarre, Spain
website: http://www.fincaalbret.com/
phone: +34 948 40 68 06
rating: 4.8
reviews: 10 Google reviews
schedule: Sunday Closed
Monday (Easter Monday)7:15 am–6 pmHours might differ
Tuesday 7:15 am–6 pm
Wednesday 7:15 am–6 pm
Thursday 7:15 am–6 pm
Friday 7:15 am–6 pm
Saturday Closed

socialmedia: NULL
status: OPEN
11993.
Lat: 42.2344168
Long: -1.7060147
Asta nu cere
11994. https://www.google.com/search?q=Finca%20Allende%20winery%20in%20Rioja%20%20Spain%20%20Northern%20Spain
11994. Finca Allende
adress: Pl. Ibarra, 1, 26330 Briones, La Rioja, Spain
website: http://www.finca-allende.com/
phone: +34 941 32 23 01
rating: 4.1
reviews: 28 Google reviews
schedule: Sunday Closed
Monday (Easter Monday)9 am–6 pmHours might differ
Tuesday 9 am–6 pm
Wednesday 9 am–6 pm
Thursday 9 am

In [6]:
index

[13200, 13447, 13602, 13764, 13895]

## Saving The Data

In [8]:
finish

'12000'

In [6]:
with open('From'+finish+'Good.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(list_of_google_wineries)

In [7]:
with open('From'+finish+'Bad.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header2)
        writer.writerows(list_of_non_google_wineries)

## Bug Repairing

In [9]:
BRdf = pd.read_csv(r"Path to csv")
BRdf1 = pd.read_csv(r"Path to csv"+finish+"Bad.csv")
BRdf2 = pd.read_csv(r"Path to csv"+finish+"Good.csv")
col_one_list = BRdf1['winery'].tolist()

In [10]:
names = []
regions = []
l = 0
for i in range(int(start),int(finish)):
    if(BRdf.winery[i] not in col_one_list):
        l = l+1
        names.append(BRdf.winery[i])
        regions.append(BRdf.region[i])
print(l)

1240


In [11]:
for i in range(len(BRdf2.winery)):
    print(i)
    print(BRdf2.winery[i])
    print(names[i])

0
Domaine de Coste Chaude
Domaine de Coste Chaude
1
Domaine de Courcel
Domaine de Courcel
2
Domaine de Couron
Domaine de Couron
3
Domaine de Courteillac
Domaine de Courteillac
4
Domaine de Coussergues
Domaine de Coussergues
5
Domaine de Curebeasse
Domaine de Curebeasse
6
Domaine de Cébène
Domaine de Cébène
7
Domaine de Côtes Rémont
Domaine de Côtes Rémont
8
Domaine de Dauliac
Domaine de Dauliac
9
Domaine de Deurre
Domaine de Deurre
10
Domaine de Dionysos
Domaine de Dionysos
11
Domaine de Durban
Domaine de Durban
12
Domaine de Fabrègues
Domaine de Fabrègues
13
Domaine de Familongue
Domaine de Familongue
14
Domaine de Fauterie
Domaine de Fauterie
15
Domaine de Fenouillet
Domaine de Fenouillet
16
Domaine de Ferrand
Domaine de Ferrand
17
Domaine de Fondrèche
Domaine de Fondrèche
18
Domaine de Fonfile
Domaine de Fonfile
19
Domaine de Fontavin
Domaine de Fontavin
20
Domaine de Fontenille
Domaine de Fontenille
21
Domaine de Foretal
Domaine de Foretal
22
Domaine de Frégate
Domaine de Frégate
2

In [3]:
for i in range(len(BRdf2.winery)):
    BRdf2.winery[i] = names[i]
    BRdf2.region[i] = regions[i]

NameError: name 'BRdf2' is not defined

In [4]:
for i in range(len(BRdf2.winery)):
    try:
        if math.isnan(BRdf2.region[i]):
            BRdf2.region[i] = " "
        BRdf2.GoogleUrl[i] = "https://www.google.com/search?q=" + (BRdf2.winery[i] + " winery in " + BRdf2.region[i].split(',')[0] + " "+ BRdf2.region[i].split(',')[BRdf2.region[i].count(',')] + " " + BRdf2.region[i].split(',')[BRdf2.region[i].count(',')-1]).replace('&','%26').replace(' ','%20')
    except:
        BRdf2.GoogleUrl[i] = "https://www.google.com/search?q=" + (BRdf2.winery[i] + " winery in " + BRdf2.region[i].split(',')[0] + " "+ BRdf2.region[i].split(',')[BRdf2.region[i].count(',')] + " " + BRdf2.region[i].split(',')[BRdf2.region[i].count(',')-1]).replace('&','%26').replace(' ','%20')

NameError: name 'BRdf2' is not defined

In [14]:
BRdf2.to_csv('From'+finish+'GoodDebug.csv', index =False, encoding='utf-8-sig')

## Adding the days columns

In [15]:
ADdf2 = pd.read_csv(r"Path to csv"+finish+"GoodDebug.csv")
results = ADdf2

In [2]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
for day in days:
    results[str(day)] = " "
for i in range(len(results.schedule)):
    try:
        for result in results.schedule[i].split("\n"):
            if result.split(" ")[0] in days:
                print(result[len(result.split(" ")[0]):])
                if len(result[len(result.split(" ")[0]):])==0:
                    results[str(result.split(" ")[0])][i] = "NULL"
                else:
                    results[str(result.split(" ")[0])][i] = result[len(result.split(" ")[0]):]
    except:
        for day in days:
            results[str(day)][i] = "NULL"

NameError: name 'results' is not defined

In [17]:
results.to_csv('From'+finish+'GoodDebugAD.csv', index =False, encoding='utf-8-sig')

## Rechecking for Long and Lat

In [18]:
RLLdf = pd.read_csv(r"Path to csv"+finish+"GoodDebugAD.csv")

In [19]:
namesWineriesToRetry = []
for i in index:
    namesWineriesToRetry.append(df.winery[i])

In [20]:
index = []
for i in range(len(RLLdf.winery)):
    if RLLdf.winery[i] in namesWineriesToRetry:
        index.append(i)

In [21]:
index

[268,
 540,
 595,
 598,
 638,
 641,
 645,
 719,
 738,
 773,
 874,
 880,
 893,
 931,
 976,
 985,
 1220,
 1223]

In [22]:
%%time
index2 = []
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
for i in index:
    driver.get(RLLdf.GoogleMapsUrl[i])
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR ,'#yDmH0d > c-wiz > div > div > div > div.NIoIEf > div.G4njw > div.AIC7ge > div.CxJub > div.VtwTSb > form:nth-child(1) > div > div > button > span'))).click()
    except:
        pass
    time.sleep(5)
    print(i)
    print(driver.current_url)
    print(driver.current_url[driver.current_url.find("!1d")+3:driver.current_url.find("!2d")])
    print(driver.current_url[driver.current_url.find("!2d")+3:])
    if driver.current_url[driver.current_url.find("!1d")+3:driver.current_url.find("!2d")].startswith("tps://www")==0:
        RLLdf.longitude[i] = driver.current_url[driver.current_url.find("!1d")+3:driver.current_url.find("!2d")]
        RLLdf.latitude[i] = driver.current_url[driver.current_url.find("!2d")+3:]
        index2.append(i)
for j in index2:
    index.remove(j)

268
https://www.google.com/maps/dir//Charavin+Didier+Route+Stade,+84110+Rasteau,+France/@54.5512799,-4.4737716,5z
tps://www.google.com/maps/dir//Charavin+Didier+Route+Stade,+84110+Rasteau,+France/@54.5512799,-4.4737716,5
tps://www.google.com/maps/dir//Charavin+Didier+Route+Stade,+84110+Rasteau,+France/@54.5512799,-4.4737716,5z
540
https://www.google.com/maps/dir//Dominio+de+Pingus++34+983+68+01+89/@54.5512799,-4.4737716,5z
tps://www.google.com/maps/dir//Dominio+de+Pingus++34+983+68+01+89/@54.5512799,-4.4737716,5
tps://www.google.com/maps/dir//Dominio+de+Pingus++34+983+68+01+89/@54.5512799,-4.4737716,5z
595
https://www.google.com/maps/dir//Finca+Dos+Puentes++54+9+261+684-7018/@54.5512799,-4.4737716,5z
tps://www.google.com/maps/dir//Finca+Dos+Puentes++54+9+261+684-7018/@54.5512799,-4.4737716,5
tps://www.google.com/maps/dir//Finca+Dos+Puentes++54+9+261+684-7018/@54.5512799,-4.4737716,5z
598
https://www.google.com/maps/dir//Double+Canyon++1+509-579-7065/@54.5512799,-4.4737716,5z
tps://www.

In [23]:
RLLdf.to_csv("From"+str(finish)+"GoodDebugADRll.csv", index = False,encoding='utf-8-sig')

## Transfering the ones that start with tps://

In [24]:
newindex = index

In [25]:
TFdf = pd.read_csv(r"Path to csv"+finish+"GoodDebugADRll.csv")

In [26]:
TFdf2 = []
for i in index:
    row = [TFdf.winery[i],TFdf.region[i]]
    TFdf2.append(row)

In [27]:
header2 = ["winery","region"]
with open('From'+finish+'NewBad.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header2)
        writer.writerows(TFdf2)

In [28]:
TFdf = TFdf.drop(index=newindex, axis = 0)
TFdf = TFdf.reset_index(drop=True)

In [29]:
TFdf.to_csv("From"+finish+"GoodDebugRllTF.csv", index =False, encoding='utf-8-sig')

## Dropping the Google,Maps, Schedule columns

In [30]:
Dropdf = pd.read_csv(r"Path to csv"+finish+"GoodDebugRllTF.csv")

In [31]:
Dropdf.drop("schedule",inplace=True,axis=1)
Dropdf.drop("GoogleUrl",inplace=True,axis=1)
Dropdf.drop("GoogleMapsUrl",inplace=True,axis=1)

In [32]:
Dropdf.to_csv("From"+finish+"GoodDebugRllTFDrop.csv", index =False, encoding='utf-8-sig')

## Checking for similarity

In [33]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [34]:
CSdf = pd.read_csv(r"Path to csv"+str(finish)+"GoodDebugRllTFDrop.csv")

In [35]:
CSdf2=[]
count=0
for i in range(len(CSdf.winery)):
    if(similar(CSdf.winery[i].lower(),CSdf["google title"][i].lower()) < 0.30):
        for j in CSdf.winery[i].split(" "):
            if CSdf["google title"][i].find(j) ==-1:
                count = count+1
                print(CSdf.winery[i])
                print(CSdf["google title"][i])
                print(similar(CSdf.winery[i],CSdf["google title"][i]))
                CSdf2.append(CSdf.iloc[i])
                break

Domaine de Curebeasse
Château Paquette
0.21621621621621623
Domaine de Familongue
SCEA Quinquarlet
0.16216216216216217
Domaine de Foretal
Perraud Jean-Yves
0.17142857142857143
Domaine de Moulin-Pouzy
Vignobles Fabien Castaing
0.16666666666666666
Domaine de Viranel
VIRANEL (Vin / Wine) - AOP Saint-Chinian - Languedoc - Rouge Rosé Blanc
0.1797752808988764
Domaine de l'Harmas
Brézème
0.15384615384615385
Domaine de la Chapelle
Pascal Rollet
0.22857142857142856
Domaine de la Chapelle Saint-Victor
Chateau du Seuil
0.27450980392156865
Domaine de la Petite Roche
Régnard
0.12121212121212122
Domaine des Foudres
Château du Moulin-à-Vent
0.18604651162790697
Domaine des Hauts Plateaux
Château La Coste
0.19047619047619047
Domaine du Roucas de St Pierre
Maison Gabriel Meffre - Caveau au Domaine de Longue Toque
0.2988505747126437
Domaine la Garrigue
Bernard Albert
0.24242424242424243
Domaine les Hautes Noëlles
Juliane et Jean-Gabriel Tridon
0.25
Domaine les Pins
SCEV PITAULT-LANDRY & FILS
0.09523809523

In [36]:
header = ["winery","region","google title","adress","website","phone","rating","reviews","socialmedia","status","latitude","longitude","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
with open('From'+str(finish)+'GoodDebugRllTFDropNotSure.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(CSdf2)

## Separating the not similar

In [37]:
DSdf = pd.read_csv(r"Path to csv"+str(finish)+"GoodDebugRllTFDrop.csv")
DSdf1 = pd.read_csv(r"Path to csv"+str(finish)+"GoodDebugRllTFDropNotSure.csv")
col_one_list = DSdf1['winery'].tolist()

In [38]:
count = 0
DSdf2 = []
for i in range(len(DSdf.winery)):
    if(DSdf.winery[i] not in col_one_list):
        print(DSdf.winery[i])
        DSdf2.append(DSdf.iloc[i])
        count= count+1
print(count)

Domaine de Coste Chaude
Domaine de Courcel
Domaine de Couron
Domaine de Courteillac
Domaine de Coussergues
Domaine de Cébène
Domaine de Côtes Rémont
Domaine de Dauliac
Domaine de Deurre
Domaine de Dionysos
Domaine de Durban
Domaine de Fabrègues
Domaine de Fauterie
Domaine de Fenouillet
Domaine de Ferrand
Domaine de Fondrèche
Domaine de Fonfile
Domaine de Fontavin
Domaine de Fontenille
Domaine de Frégate
Domaine de Galuval
Domaine de Gensac
Domaine de Givaudan
Domaine de Gournier
Domaine de Granajolo
Domaine de Grand Garant
Domaine de Grandmaison
Domaine de Grange Neuve
Domaine de Grangeneuve
Domaine de Gry-Sablon
Domaine de Guillaman
Domaine de Guiot
Domaine de Jarras
Domaine de Joÿ
Domaine de Juchepie
Domaine de L'Olivier
Domaine de La Tourbeille
Domaine de Laballe
Domaine de Lalande
Domaine de Lamothe
Domaine de Lauzières
Domaine de Laxé
Domaine de Leyre-Loup
Domaine de Longue Toque
Domaine de Longval
Domaine de Lumian
Domaine de Magalanne
Domaine de Magnaut
Domaine de Maison Neuve
D

In [39]:
header = ["winery","region","google title","adress","website","phone","rating","reviews","socialmedia","status","latitude","longitude","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
with open('From'+finish+'GoodDebugRllTFDropGood.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(DSdf2)

In [43]:
driver.close()